# ePSF Fitting

This version of TGLC includes generic functions that can be used to fit an ePSF to an image with known star positions and brightnesses. This tutorial simulates an FFI cutout and then walks fitting an ePSF for the cutout.

To install the notebook dependencies, run the following in your python environment (a virtual environment is a good idea!):

```shell
pip install astropy matplotlib numpy jupyter photutils "tglc @ git+ssh://github.com/mit-kavli-institute/tess-gaia-light-curve"
```


In [ ]:
from astropy.visualization import simple_norm
import matplotlib.pyplot as plt
import numpy as np
from photutils.datasets import make_model_image, make_model_params, make_noise_image
from photutils.psf import CircularGaussianPSF

from tglc.epsf import fit_epsf, make_tglc_design_matrix
from tglc.utils.constants import convert_tess_magnitude_to_tess_flux


## Create simulated image

First we create a simulated 150x150 image, like the cutouts TGLC uses from actual TESS FFIs. We use a circular Gaussian PSF with FWHM 1.5 and 1000 sources with Tmag <= 13, in an exponential distribution favoring dim stars.


In [ ]:
seed = 3141592

In [ ]:
model = CircularGaussianPSF()
shape = (150, 150)
n_sources = 1000
params = make_model_params(
    shape,
    n_sources,
    x_name="x_0",
    y_name="y_0",
    fwhm=(1.5, 1.5),
    seed=seed,
)
# Update flux values to reflect true Tmag distribution
rng = np.random.default_rng(seed)
tmag = 13 - rng.exponential(0.75, n_sources)
params["flux"] = convert_tess_magnitude_to_tess_flux(tmag).value

In [ ]:
plt.xscale("log")
plt.hist(params["flux"], bins=200)
plt.xlabel("Flux")
plt.ylabel("Number of sources")
plt.title("Flux Distribution (log scale)")

In [ ]:
model_shape = (11, 11)
simulated_cutout = make_model_image(
    shape, model, params, model_shape=model_shape, x_name="x_0", y_name="y_0"
)

noise = make_noise_image(shape, distribution="gaussian", mean=5, stddev=2, seed=seed)
simulated_cutout += noise

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
norm = simple_norm(simulated_cutout, "sqrt", percent=99)
ax.imshow(simulated_cutout, norm=norm, origin="lower")
ax.scatter(params["x_0"], params["y_0"], s=1, marker=".", color="red")
ax.set(xlim=(0, shape[1]), ylim=(0, shape[0]))
ax.set_title("Simulated image")

## Fitting an ePSF

Now, we fit an ePSF for the simulated cutout using TGLC's linear methods. We can choose a shape `(sy, sx)` of the PSF model in pixel coordinates, and an oversampling factor `f`. Then evaluating the PSF amounts to linearly interpolating a grid with shape `(sy * f + 1, sx * f + 1)`. There are also 6 parameters used to describe the background: 3 for a linear gradient background and 3 for a spatially-varying contribution from TESS's CCD "straps." We will ignore the background strap contributions for now, but TGLC's package data does include FITS files with the strap data for each CCD.

To run the ePSF fit, we flatten the `(sy * f + 1, sx * f + 1)` PSF grid into a column vector of parameters, plus the 6 background parameters. We also flatten the cutout into a column vector as our observed data. We then create a design matrix `M` where `M[r, c]` gives the contribution of model parameter `r` to observed data `c`. So, if `p` is our column vector of model parameters and `o` is our column vector of observed data, we are running a least squares fit for the equation `Mp =~ o`.

For proper regularization, the design matrix for running fits has additional data compared to the design matrix described above. This extra data is added when `edge_compression_scale_factor` is specified in `make_tglc_design_matrix`; when it is omitted, the design matrix produced is as described above. Note that `make_tglc_design_matrix` can also make a design matrix that omits the background parameters, which happens if `background_strap_mask` is not specified - this is why we set `background_strap_mask` to all zeros in the code below rather than omitting it.

Because the design matrix used for fitting has extra information, it is not appropriate for forward modeling once we have obtained the optimal model parameters. Therefore, we use a new design matix to forward model the cutout.


In [ ]:
star_positions = np.column_stack([params["x_0"].data, params["y_0"].data])
flux_ratios = params["flux"] / max(params["flux"])
psf_shape_pixels = (11, 11)
oversample_factor = 2

# Note: the make_tglc_design_matrix function is just-in-time (JIT) compiled, so it will be slow on
# the first time it is called but much faster after that
fit_design_matrix, regularization_dimensions = make_tglc_design_matrix(
    shape,
    psf_shape_pixels,
    oversample_factor,
    star_positions,
    flux_ratios,
    background_strap_mask=np.zeros(shape),  # no contribution from CCD straps
    edge_compression_scale_factor=1e-4,
)
epsf = fit_epsf(
    fit_design_matrix, simulated_cutout, np.zeros(shape, dtype=bool), 1.4, regularization_dimensions
)

In [ ]:
plt.imshow(epsf[:-6].reshape(23, 23), extent=(-11.5, 11.5, -11.5, 11.5), origin="lower")
plt.scatter(0, 0, marker=".", color="red")
plt.title("ePSF")

In [ ]:
forward_model_design_matrix, _ = make_tglc_design_matrix(
    shape, (11, 11), 2, star_positions, flux_ratios, np.zeros(shape)
)
model_data = (forward_model_design_matrix @ epsf).reshape(shape)

In [ ]:
norm = simple_norm(np.stack([simulated_cutout, model_data]), "sqrt", percent=99)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(simulated_cutout, norm=norm, origin="lower")
ax1.scatter(params["x_0"], params["y_0"], s=1, marker=".", color="red")
ax1.set(xlim=(-0.5, shape[1] - 0.5), ylim=(-0.5, shape[0] - 0.5))
ax1.set_title("Simulated image")

ax2.imshow(model_data, norm=norm, origin="lower")
ax2.scatter(params["x_0"], params["y_0"], s=1, marker=".", color="red")
ax2.set(xlim=(-0.5, shape[1] - 0.5), ylim=(-0.5, shape[0] - 0.5))
ax2.set_title("Image with ePSF model")